# 📊 Week 6 — Session 1: Data Wrangling and Aggregation

## Transform, Aggregate, Analyze!

**Data wrangling** is the process of transforming raw data into a more useful format. Combined with **aggregation**, you can summarize large datasets into meaningful insights.

---

### 🎯 Learning Objectives

By the end of this session, you will be able to:

- ✅ Transform data using `map()`, `filter()`, and list comprehensions
- ✅ Group data using `groupby()` in pandas
- ✅ Apply aggregation functions: `sum()`, `mean()`, `count()`, `min()`, `max()`
- ✅ Create pivot tables and cross-tabulations
- ✅ Chain multiple operations together

### 📋 Key Concepts Overview

| Concept | Purpose | Example |
|---------|---------|---------|
| **Wrangling** | Reshape & transform data | Filter rows, select columns, create new columns |
| **Aggregation** | Summarize data | Calculate totals, averages, counts |
| **Grouping** | Split-apply-combine | Sum sales by region |
| **Pivoting** | Reshape for analysis | Rows to columns |

**💡 Real-world use:** "What are total sales by region and product category?"

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚀 SETUP: Import required libraries
# ═══════════════════════════════════════════════════════════════════════════════
import pandas as pd
import numpy as np

print("✅ Libraries imported!")
print(f"📦 Pandas version: {pd.__version__}")

---

## 📊 Section 1: Creating Sample Data for Practice

Let's create a realistic sales dataset to work with throughout this session.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 CREATE SAMPLE SALES DATA
# ═══════════════════════════════════════════════════════════════════════════════

# Create a sales dataset
sales_data = pd.DataFrame({
    'order_id': range(1001, 1021),
    'date': pd.date_range('2024-01-01', periods=20, freq='D'),
    'region': ['North', 'South', 'East', 'West', 'North'] * 4,
    'category': ['Electronics', 'Clothing', 'Electronics', 'Home', 'Clothing',
                 'Electronics', 'Home', 'Clothing', 'Home', 'Electronics'] * 2,
    'product': ['Laptop', 'Shirt', 'Phone', 'Lamp', 'Pants',
                'Tablet', 'Chair', 'Jacket', 'Table', 'Monitor'] * 2,
    'quantity': [1, 3, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 3, 2, 1, 1, 2, 1, 1, 3],
    'unit_price': [999, 29, 699, 45, 49, 449, 199, 89, 299, 349,
                   999, 29, 699, 45, 49, 449, 199, 89, 299, 349],
    'salesperson': ['Alice', 'Bob', 'Charlie', 'Diana', 'Alice',
                    'Bob', 'Charlie', 'Diana', 'Alice', 'Bob'] * 2
})

# Calculate total
sales_data['total'] = sales_data['quantity'] * sales_data['unit_price']

print("📊 SAMPLE SALES DATA")
print("=" * 70)
print(sales_data.head(10))
print(f"\n📐 Shape: {sales_data.shape}")
print(f"📊 Total Revenue: ${sales_data['total'].sum():,.2f}")

---

## 🔧 Section 2: Data Selection and Filtering

Before aggregating, you need to know how to select and filter data. These are the building blocks of data wrangling.

### 📋 Selection Methods

| Method | Purpose | Example |
|--------|---------|---------|
| `df['col']` | Select single column | `df['region']` |
| `df[['col1', 'col2']]` | Select multiple columns | `df[['region', 'total']]` |
| `df.loc[]` | Select by label | `df.loc[0:5, 'region']` |
| `df.iloc[]` | Select by position | `df.iloc[0:5, 0]` |
| `df[condition]` | Filter rows | `df[df['total'] > 500]` |

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚫 BAD PRACTICE: Chained indexing (can cause warnings & issues)
# ═══════════════════════════════════════════════════════════════════════════════

print("🚫 BAD: Chained indexing")
print("-" * 40)

# DON'T do this - it can cause SettingWithCopyWarning
# df[df['region'] == 'North']['total'] = 0  # This might not work!

print("❌ Problem: df[condition]['column'] = value")
print("   This creates a copy, not a view!")
print("   Your changes might not be saved!")
print("=" * 50)

# ═══════════════════════════════════════════════════════════════════════════════
# 🌱 NOVICE: Basic selection and filtering
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Simple, clear selections

print("🌱 NOVICE: Basic selection and filtering")
print("-" * 40)

df = sales_data.copy()

# Select a single column
print("📋 Regions:")
print(df['region'].unique())

# Filter rows
north_sales = df[df['region'] == 'North']
print(f"\n📊 North region sales: {len(north_sales)} orders")

# Multiple conditions with & (and) and | (or)
big_electronics = df[(df['category'] == 'Electronics') & (df['total'] > 500)]
print(f"\n📊 Big Electronics orders (>$500): {len(big_electronics)}")
print(big_electronics[['product', 'total']])
print("=" * 50)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✨ PYTHONIC: Using .loc for safe selection
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Use .loc for explicit, safe selection

print("✨ PYTHONIC: Using .loc for selection")
print("-" * 40)

df = sales_data.copy()

# Select specific rows and columns with .loc
print("📋 First 5 orders (region and total only):")
print(df.loc[0:4, ['region', 'total']])

# Filter and select in one step
print("\n📊 South region sales details:")
south_sales = df.loc[df['region'] == 'South', ['product', 'quantity', 'total']]
print(south_sales)

# Using .query() for readable filtering
print("\n📊 Using .query() - more readable:")
high_value = df.query('total > 500 and category == "Electronics"')
print(high_value[['product', 'category', 'total']])
print("=" * 50)

---

## 📈 Section 3: Aggregation Fundamentals

**Aggregation** reduces many values to one summary value. This is essential for data analysis!

### 📋 Common Aggregation Functions

| Function | Purpose | Example |
|----------|---------|---------|
| `sum()` | Total | Total sales |
| `mean()` | Average | Average order value |
| `count()` | Count | Number of orders |
| `min()` / `max()` | Extremes | Smallest/largest order |
| `median()` | Middle value | Typical order |
| `std()` | Spread | How varied are orders? |

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚫 BAD PRACTICE: Manual aggregation loops
# ═══════════════════════════════════════════════════════════════════════════════

print("🚫 BAD: Manual aggregation with loops")
print("-" * 40)

# DON'T do this - it's slow and error-prone
total = 0
for i in range(len(sales_data)):
    total += sales_data.iloc[i]['total']
print(f"Manual sum: ${total:,.2f}")

# Counting manually
count = 0
for i in range(len(sales_data)):
    if sales_data.iloc[i]['region'] == 'North':
        count += 1
print(f"Manual count of North: {count}")

print("\n❌ Problems:")
print("   1. Slow with large datasets")
print("   2. More code = more bugs")
print("   3. Hard to read")
print("=" * 50)

# ═══════════════════════════════════════════════════════════════════════════════
# 🌱 NOVICE: Basic aggregation functions
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Use built-in pandas methods

print("🌱 NOVICE: Basic aggregation")
print("-" * 40)

df = sales_data.copy()

# Simple aggregations
print(f"📊 Total Revenue: ${df['total'].sum():,.2f}")
print(f"📊 Average Order: ${df['total'].mean():,.2f}")
print(f"📊 Order Count: {df['total'].count()}")
print(f"📊 Min Order: ${df['total'].min():,.2f}")
print(f"📊 Max Order: ${df['total'].max():,.2f}")
print(f"📊 Median Order: ${df['total'].median():,.2f}")
print("=" * 50)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 INTERMEDIATE: Multiple aggregations with .agg()
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Apply multiple aggregations at once

print("🔧 INTERMEDIATE: Multiple aggregations with .agg()")
print("-" * 40)

df = sales_data.copy()

# Multiple aggregations on one column
order_stats = df['total'].agg(['sum', 'mean', 'min', 'max', 'count'])
print("📊 Order Statistics:")
print(order_stats)

# Multiple aggregations on multiple columns
multi_stats = df.agg({
    'total': ['sum', 'mean'],
    'quantity': ['sum', 'mean']
})
print("\n📊 Multi-column Statistics:")
print(multi_stats)
print("=" * 50)

---

## 🔀 Section 4: GroupBy - Split-Apply-Combine

The **groupby** operation is the heart of data aggregation. It follows the **split-apply-combine** pattern:

1. **Split** data into groups
2. **Apply** a function to each group
3. **Combine** results back together

### 📊 Visual Example
```
DataFrame → Split by Region → Apply sum() → Combine
                North → $5000
Sales Data →    South → $3000  → Summary Table
                East  → $4000
                West  → $2000
```

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚫 BAD PRACTICE: Manual grouping with loops
# ═══════════════════════════════════════════════════════════════════════════════

print("🚫 BAD: Manual grouping")
print("-" * 40)

# DON'T do this
region_totals = {}
for i in range(len(sales_data)):
    region = sales_data.iloc[i]['region']
    total = sales_data.iloc[i]['total']
    if region not in region_totals:
        region_totals[region] = 0
    region_totals[region] += total

print("Manual grouping result:")
for region, total in region_totals.items():
    print(f"   {region}: ${total:,.2f}")

print("\n❌ Problems: Slow, verbose, error-prone")
print("=" * 50)

# ═══════════════════════════════════════════════════════════════════════════════
# 🌱 NOVICE: Basic groupby operations
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Use pandas groupby for easy grouping

print("🌱 NOVICE: Basic groupby")
print("-" * 40)

df = sales_data.copy()

# Group by region, sum the totals
region_sales = df.groupby('region')['total'].sum()
print("📊 Sales by Region:")
print(region_sales)

# Group by category
category_sales = df.groupby('category')['total'].sum()
print("\n📊 Sales by Category:")
print(category_sales)

# Count orders per salesperson
orders_per_person = df.groupby('salesperson')['order_id'].count()
print("\n📊 Orders per Salesperson:")
print(orders_per_person)
print("=" * 50)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 INTERMEDIATE: Multiple groupby aggregations
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Apply multiple functions to grouped data

print("🔧 INTERMEDIATE: Multiple groupby aggregations")
print("-" * 40)

df = sales_data.copy()

# Multiple aggregations per group
region_stats = df.groupby('region').agg({
    'total': ['sum', 'mean', 'count'],
    'quantity': 'sum'
})
print("📊 Detailed Region Statistics:")
print(region_stats)

# Group by multiple columns
region_category = df.groupby(['region', 'category'])['total'].sum()
print("\n📊 Sales by Region and Category:")
print(region_category)
print("=" * 50)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✨ PYTHONIC: Named aggregations with reset_index()
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Clean, readable named aggregations

print("✨ PYTHONIC: Named aggregations")
print("-" * 40)

df = sales_data.copy()

# Named aggregations (pandas 0.25+)
summary = df.groupby('region').agg(
    total_revenue=('total', 'sum'),
    avg_order=('total', 'mean'),
    num_orders=('order_id', 'count'),
    total_items=('quantity', 'sum')
).reset_index()  # Convert index back to column

print("📊 Clean Summary Table:")
print(summary)

# Sort by total revenue
summary_sorted = summary.sort_values('total_revenue', ascending=False)
print("\n📊 Sorted by Revenue (descending):")
print(summary_sorted)
print("=" * 50)

---

## 📊 Section 5: Pivot Tables

**Pivot tables** reshape your data to show summaries from different perspectives. They're like Excel pivot tables!

### 📋 Pivot Table Parameters

| Parameter | Purpose |
|-----------|---------|
| `values` | Column to aggregate |
| `index` | Row grouping |
| `columns` | Column grouping |
| `aggfunc` | Aggregation function |

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✨ PYTHONIC: Pivot Tables
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Create Excel-like pivot tables

print("✨ PYTHONIC: Pivot Tables")
print("-" * 40)

df = sales_data.copy()

# Basic pivot: Region vs Category with total sales
pivot = df.pivot_table(
    values='total',
    index='region',
    columns='category',
    aggfunc='sum',
    fill_value=0  # Fill missing combinations with 0
)

print("📊 Sales Pivot: Region x Category")
print(pivot)

# Add totals
pivot['Total'] = pivot.sum(axis=1)
print("\n📊 With Row Totals:")
print(pivot)

# Multiple aggregations
pivot_detailed = df.pivot_table(
    values='total',
    index='region',
    columns='category',
    aggfunc=['sum', 'mean', 'count'],
    fill_value=0
)
print("\n📊 Detailed Pivot (sum, mean, count):")
print(pivot_detailed)
print("=" * 50)

---

## 🎯 Section 6: Interactive Practice Activities

### 🏋️ Activity 1: Sales Analysis
Answer these questions about our sales data:

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 ACTIVITY 1: Sales Analysis Questions
# ═══════════════════════════════════════════════════════════════════════════════
# Answer these questions using groupby and aggregation!

df = sales_data.copy()

# Q1: What is the total revenue by salesperson?
q1_answer = df.groupby('salesperson')['total'].sum()
print("Q1: Revenue by Salesperson:")
print(q1_answer)

# Q2: What is the average order value by category?
q2_answer = df.groupby('category')['total'].mean()
print("\nQ2: Average Order by Category:")
print(q2_answer)

# Q3: Which region has the most orders?
q3_answer = df.groupby('region')['order_id'].count().sort_values(ascending=False)
print("\nQ3: Orders by Region:")
print(q3_answer)

# Q4: TODO - Find the top-selling product by quantity
# Hint: groupby('product')['quantity'].sum()
q4_answer = None  # Your code here

# Q5: TODO - Calculate average revenue per salesperson per region
# Hint: groupby(['salesperson', 'region'])
q5_answer = None  # Your code here

---

## 📝 Section 7: Key Takeaways & Summary

### 🎯 What You Learned Today

| Concept | Key Methods |
|---------|-------------|
| **Selection** | `df['col']`, `df.loc[]`, `df.query()` |
| **Filtering** | Boolean indexing, `&`, `\|` |
| **Aggregation** | `sum()`, `mean()`, `count()`, `agg()` |
| **GroupBy** | `df.groupby('col').agg()` |
| **Pivot Tables** | `df.pivot_table()` |

### 💡 Best Practices Checklist

✅ Use `.loc[]` for safe selection  
✅ Use `.query()` for readable filtering  
✅ Use named aggregations for clear output  
✅ Always call `.reset_index()` if you need a regular DataFrame  
✅ Use pivot tables for multi-dimensional summaries  

### 🔮 What's Next?

In the group session, we'll practice these skills together on a more complex dataset!

---

**🏆 Great job completing Week 6 Session 1!**